In [0]:
import nltk as ntk
import pandas as pd
import numpy as np
import string
import re
from google.colab import files
import io
import nltk.corpus
import os
from nltk.tokenize import  word_tokenize
import csv
import spacy
import nltk as ntk
import tensorflow_hub as hub
import tensorflow as tf

In [0]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# this will give a link to your google drive, please click on the link and follow steps, which will give the verification code
# once verification code validated then , you should get the sharable link of the file and copy it in the below section

In [0]:
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate
from sklearn import preprocessing, model_selection

In [0]:
link = 'https://drive.google.com/open?id=1U2_iClctC8So0Y1J-GHAyXj6AXlEscJg' 
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('drugsComTrain_raw.csv')  
fMyfile = pd.read_csv('drugsComTrain_raw.csv')

In [0]:
fMyfile = fMyfile[pd.notnull(fMyfile['condition'])]
fMyfile.condition =fMyfile.condition.str.replace(r"[^a-z'A-Z]+", " ")
fMyfile.condition=fMyfile.condition.str.replace('<[^<]+?>', '')
# replace those values in condition columns which has aggreagetd count less than 10
s=fMyfile['condition'].value_counts()
fMyfile['condition1'] = np.where(fMyfile['condition'].isin(s.index[s >=10]), fMyfile['condition'], 'Other')

In [75]:
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
nltk.download('stopwords')
nltk.download('punkt')

fMyfile.review = fMyfile.review.str.replace('&#039;', "'")
fMyfile.review = fMyfile.review.str.replace('&\w+;',' ')
fMyfile.review = fMyfile.review.str.replace(r"[^a-z'A-Z]+", " ")

stop_words = set(stopwords.words('english')) 
def Remove_UnnecessaryWords(ReviewLine):
  finalreview=" "
  review = ReviewLine['review']
  tokens = nltk.word_tokenize(review)
  token_words = [w for w in tokens if w.isalpha()]
  meaningful_words = [w for w in token_words if not w in stop_words]
  finalreview=finalreview.join(meaningful_words)
  return(finalreview)
fMyfile['review'] = fMyfile.apply(Remove_UnnecessaryWords, axis=1)



[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:
from numpy import array
from numpy import asarray
from numpy import zeros
link = "https://drive.google.com/open?id=1BpWHnVTX8EBU8-p34KhZtdw1wKT5WeBN"
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('glove.6B.100d.txt')  
glove_file  = open("glove.6B.100d.txt",'r', encoding='utf-8')
embeddings_dictionary = dict()

In [77]:
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English
nlp = spacy.load('en')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = max(fMyfile['review'].apply(len))
MAX_SEQUENCE_LENGTH=100
tokenizer = Tokenizer(num_words=MAX_NB_WORDS, filters='!"#$%&()*+,-./:;<=>?@[\]^_`{|}~', lower=True)
tokenizer.fit_on_texts(fMyfile['review'].values)
vocab_size = len(tokenizer.word_index) + 1
word_index = tokenizer.word_index
X = tokenizer.texts_to_sequences(fMyfile['review'].values)
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
print(X.shape)

(160398, 100)


In [101]:
for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary [word] = vector_dimensions
glove_file.close()
embedding_matrix = zeros((vocab_size, 100))
print(embedding_matrix.shape)
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    try:
      if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector
    except:
      pass

(45817, 100)


In [0]:

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

y=fMyfile['condition1']
y=pd.get_dummies(y)
from keras import utils as np_utils

train_x, test_x, train_y, test_y = model_selection.train_test_split(X,y,test_size=0.2)

In [0]:

model = Sequential()

#model.add(Embedding(input_dim=vocab_size, output_dim=8, input_length=100))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=100 , trainable=False)
model.add(embedding_layer)
model.add(Flatten())
#model.add(LSTM(128))
model.add(Dense(128, activation='softmax'))
model.add(Dense(414, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

model.fit(train_x, train_y, epochs =5, batch_size =50)

scores = model.evaluate(test_x, test_y)
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Epoch 1/5
128318/128318 [==============================] - 64s 503us/step - loss: 4.7587 - acc: 0.1802
Epoch 2/5
 28200/128318 [=====>........................] - ETA: 52s - loss: 4.3279 - acc: 0.1756

In [0]:
link = 'https://drive.google.com/open?id=1Spi_frv5FH87kIe_gvm_1GQOxXnMIHrM' 
fluff, id = link.split('=')
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('drugsComTest_raw.csv')  
Test_new = pd.read_csv('drugsComTest_raw.csv')
Xnew = tokenizer.texts_to_sequences(Test_new['review'].values)
Xnew = pad_sequences(Xnew, maxlen=MAX_SEQUENCE_LENGTH)
ynew = model.predict_classes(Xnew)

In [111]:
ct=0
for i in range(len(ynew)):
  if list(y.columns.values)[ynew[i]]==Test_new['condition'][i]:
    ct=ct+1
print(ct)
print((ct/len(ynew))*100)

9862
18.34244689952758
